In [1]:
from tensorforce.environments import Environment
from tensorforce.agents import Agent
from tensorforce.execution import Runner

In [2]:
train_env_kwargs = {'filename':'sp500.csv',
    'date_from':'2008-01-01',
    'date_to':'2017-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':100,
    #'report_point':252,
    'random_sample':True,
    'reward_function':'portfolio_value'}

train_portfolio = Environment.create(
    environment='PortfolioTF',
    **train_env_kwargs
)
_ = train_portfolio.reset()
total_timesteps = 1 * (len(train_portfolio._environment.data.date.unique())-1)


C:\Users\kbine\anaconda3\envs\tensorforce\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\kbine\anaconda3\envs\tensorforce\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
vanilla = Agent.create(
    agent='reinforce',
    environment=train_portfolio,
    max_episode_timesteps=total_timesteps,
    network=dict(type='auto',
                 rnn=10  ),
    batch_size=5
)


In [ ]:
runner = Runner(
    agent=vanilla,
    environment=train_portfolio
)

In [ ]:
print(runner.agent.get_architecture())

In [ ]:
runner.agent.get_specification()

In [ ]:
runner.run(num_episodes=10)

In [ ]:
runner.run(num_episodes=10, evaluation=True)

In [ ]:
timesteps = len(train_portfolio._environment.data.date.unique())-1
random = Agent.create(
    agent='random',
    environment=train_portfolio,
    max_episode_timesteps=timesteps,
)

In [ ]:
randomrunner = Runner(
    agent=random,
    environment=train_portfolio
)

In [ ]:
randomrunner.run(num_episodes=10)

In [ ]:
total_timesteps = 1000 * (len(train_portfolio._environment.data.date.unique())-1)
a2cagent = Agent.create(
    agent='a2c',
    environment=train_portfolio,
    max_episode_timesteps=total_timesteps,
    network=dict(type='auto',
                 rnn=252  ),
    critic_optimizer=dict(optimizer='adam'),
    batch_size=5
)

In [ ]:
a2crunner = Runner(
    agent=a2cagent,
    environment=train_portfolio
)

In [ ]:
print(a2crunner.agent.get_architecture())

In [ ]:
a2crunner.agent.get_specification()

In [ ]:
a2crunner.run(num_episodes=1000)

In [2]:
a2crunner.run(num_episodes=1000, evaluation=True, save_best_agent='.' )

##%

NameError: name 'a2crunner' is not defined

In [3]:
test_env_kwargs = {'filename':'sp500.csv',
    'date_from':'2017-01-01',
    'date_to':'2019-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':100,
    #'report_point':252,
    'random_sample':False,
    'reward_function':'portfolio_value'}

test_portfolio = Environment.create(
    environment='PortfolioTF',
    **test_env_kwargs
)
_ = test_portfolio.reset()
test_timesteps = 1 * (len(test_portfolio._environment.data.date.unique())-1)

C:\Users\kbine\anaconda3\envs\tensorforce\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\kbine\anaconda3\envs\tensorforce\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
testa2c=Agent.load(directory='.',filename='best-model',environment=test_portfolio)


In [6]:
test2crunner = Runner(
    agent=testa2c,
    environment=test_portfolio
)

In [8]:
print(test2crunner.agent.get_architecture())
test2crunner.agent.get_specification()

Policy:
    Network:  
        Conv2d(name=conv2d0, size=64, window=3, padding=same, bias=True, activation=relu)
        Conv2d(name=conv2d1, size=64, window=3, padding=same, bias=True, activation=relu)
        Pooling(name=pooling, reduction=max)
        Lstm(name=lstm, cell=lstm, size=64, horizon=252, bias=False)
    Action-distribution:
        Mean:  Linear(name=mean, size=100, bias=True, initialization_scale=0.01)
        Stddev:  Linear(name=stddev, size=100, bias=True, initialization_scale=0.01)
Baseline:
    Network:  
        Conv2d(name=conv2d0, size=64, window=3, padding=same, bias=True, activation=relu)
        Conv2d(name=conv2d1, size=64, window=3, padding=same, bias=True, activation=relu)
        Pooling(name=pooling, reduction=max)
    State-value:  Linear(name=value, size=0, bias=True)


{'agent': 'a2c',
 'states': {'type': 'float', 'shape': [100, 1, 13]},
 'actions': {'type': 'float', 'shape': [100], 'min_value': 0, 'max_value': 1},
 'batch_size': 5,
 'max_episode_timesteps': 2517000,
 'network': {'type': 'auto', 'rnn': 252},
 'use_beta_distribution': False,
 'memory': 'minimum',
 'update_frequency': 1.0,
 'learning_rate': 0.001,
 'horizon': 1,
 'discount': 0.99,
 'return_processing': None,
 'advantage_processing': None,
 'predict_terminal_values': False,
 'critic': 'auto',
 'critic_optimizer': {'optimizer': 'adam'},
 'state_preprocessing': 'linear_normalization',
 'reward_preprocessing': None,
 'exploration': 0.0,
 'variable_noise': 0.0,
 'l2_regularization': 0.0,
 'entropy_regularization': 0.0,
 'parallel_interactions': 1,
 'config': None,
 'saver': None,
 'summarizer': None,
 'tracking': None,
 'recorder': None}

In [ ]:
test2crunner.run(num_episodes=10)

Episodes:   0%|          | 0/10 [00:00, return=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

In [ ]:
a2crunner.run(num_episodes=1000, evaluation=True )


